# DiscoGAN

GAN 을 통해 교차 영역 관계를 배워봅시다
https://www.youtube.com/watch?v=9reHvktowLY

![alt text](https://pbs.twimg.com/media/C7NDNRuXgAAfePz.jpg "Logo Title Text 1")

![alt text](http://www.aimechanic.com/wp-content/uploads/2017/03/PyTorch-DiscoGAN.png "Logo Title Text 1")


교차 영역 관계는 인간에게 친숙합니다
- 정장과 구두
- 영어-불어 통역

기계에게도 친숙할 수 있을까요?
조건적 이미지 생성 문제입니다
ex1) 하나의 영역으로부터 다른 곳으로 매핑하는 함수를 찾기
ex2) 다른 영역의 이미지를 주고 한 영역의 이미지를 생성하기

오늘날 대부분의 훈련 접근방식은 인간이나
알고리즘에 의해 짝지어진 데이터를 사용합니다

라벨 없이 해봅시다 :)
여러 경우에 사용될 수 있습니다
- 게임
- 실시간 피드백 디자인


하나의 이미지를 가지고 다른 것의 스타일로 재구성하기 위해서는 무엇이 필요할까요?
- 인코더-디코더? 너무 단순합니다 (카메라 필터에 더 가깝습니다) 
- 2 개의 인코더-디코더? 역호환 스타일 옮김, 하지만 여전히 단순합니다
- 적대 환경의 2 개의 인코더-디코더? 맞습니다 :) 


In [23]:
#왜 GMM 을 사용할까요?
#왜 판별기 대신 그래프를 사용할까요?
#slim.repeat 함수를 사용합니다
#2개의 참조를 추가합니다


#파이썬 2 와 파이썬 3 의 차이를 이어줍니다
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os # 파일 저장용
import numpy as np #행렬 연산

#데이터 시각화
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#머신러닝
import tensorflow as tf

#생성된 데이터에 대한 가우시안 혼합 모델
from data_gmm import GMM_distribution, sample_GMM, plot_GMM
#데이터 분석
from data_utils import shuffle, iter_data

#진행 바
from tqdm import tqdm

#TF-Slim 는 모델을 정의하고, 훈련하고, 평가하는 텐서플로우의 가벼운 라이브러리입니다. 복잡한 네트워크를 빠르고 간결하게 정의할 수 있게 해줍니다
slim = tf.contrib.slim

# 확률적 분포의 배치를 나타내는 클래스입니다
#각 클래스는 분포를 정의하는 파라미터로 초기화 됩니다
ds = tf.contrib.distributions

#교체된 텐서의 타겟을 계산하는 새로운 그래프를 생성합니다

graph_replace = tf.contrib.graph_editor.graph_replace


In [12]:
#하이퍼파라미터
""" parameters """
n_epoch = 1000 #epcoh 의 수
batch_size  = 64
dataset_size = 512
input_dim = 2 #데이터와 라벨
latent_dim = 2 
eps_dim = 2


#판별기
n_layer_disc = 2
n_hidden_disc = 256

#생성기 
n_layer_gen = 2
n_hidden_gen= 256

#추론망 (생성기 #2)
n_layer_inf = 2
n_hidden_inf= 256


In [13]:
#DiscoGAN 폴더에 결과를 저장합니다
""" Create directory for results """
result_dir = 'results/DiscoGAN/'
directory = result_dir
if not os.path.exists(directory):
    os.makedirs(directory)

![alt text](https://image.slidesharecdn.com/sampleproject-140814212447-phpapp02/95/speaker-recognition-using-gaussian-mixture-model-2-638.jpg?cb=1408051684
 "Logo Title Text 1")
 
가우시안 혼합 모델은 모든 데이터 점들이 일정 숫자의 알 수 없는 파라미터를 가진 가우시안 분포의 혼합으로 생성되었다고 가정하는 하나의 확률론적 모델입니다.

![alt text](http://i.imgur.com/GJhzOUy.png "Logo Title Text 1")


- X = n 개의 요소를 가진 데이터 세트
- alpha = k 번째 요소의 중량조합
- sigma = 가우시안 확률 밀도 함수
- mu =  k 번째 요소의 평균
- sigma2 = k 번째 요소의 분산


In [14]:
#장난감 데이터 세트로 된 데모입니다
#5 개의 요소로 된 GMM
#가우시안 혼합 모델은 하나의 확률론적 모델로써
#모든 데이터 점들이 일정 수의
#파라미터를 알 수 없는 가우시안 분포의 혼합으로
#생성되었다고 가정합니다

# X 데이터 세트를 생성합니다 (첫 번째 데이터 세트)
#input_list 의 모든 요소에 대해 함수를 적용합니다
#lambda = 익명 함수 (i.e. 이름이 정해져 있지 않은 함수)
#5 개의 요소를 가진 numpy 배열을 생성합니다
means = map(lambda x:  np.array(x), [[0, 0],
                                     [2, 2],
                                     [-1, -1],
                                     [1, -1],
                                     [-1, 1]])

#접근 방법을 리스트로 변환합니다
means = list(means)
#표준 편차
std = 0.1
#분산 - eye 는 대각선에 1 이 배치되어있고 나머지는 모두 0 인 2차원 배열인 항등 행렬을 반환합니다.
variances = [np.eye(2) * std for _ in means]

#단서를 가지기 전 이 수치에 대한
#누군가의 예상을 표현할 확률 분포
priors = [1.0/len(means) for _ in means]

#가우시안 혼합 모델을 생성합니다
gaussian_mixture = GMM_distribution(means=means,
                                               variances=variances,
                                               priors=priors)

#GMM 을 사용한 데이터의 샘플
dataset = sample_GMM(dataset_size, means, variances, priors, sources=('features', ))

#결과를 저장합니다
save_path = result_dir + 'X_gmm_data.pdf'
#결과를 표시합니다
plot_GMM(dataset, save_path)

#데이터와 라벨을 저장합니다
X_np_data= dataset.data['samples']
X_labels = dataset.data['label']

In [15]:
# 데이터 세트 Z 를 생성합니다 (두 번째 데이터 세트)
#2 개의 요소를 가진 GMM. 
means = map(lambda x:  np.array(x), [[-1, -1],[1, 1]])
means = list(means)
std = 0.1
variances = [np.eye(2) * std for _ in means]

priors = [1.0/len(means) for _ in means]

gaussian_mixture = GMM_distribution(means=means,
                                               variances=variances,
                                               priors=priors)
dataset = sample_GMM(dataset_size, means, variances, priors, sources=('features', ))
save_path = result_dir + 'Z_gmm_data.pdf'
plot_GMM(dataset, save_path)

Z_np_data= dataset.data['samples']
Z_labels = dataset.data['label']

In [16]:
# x 와 z 의 샘플
X_dataset = X_np_data
Z_dataset = Z_np_data

![alt text](https://pbs.twimg.com/media/C8QiTe2XcAATDfn.jpg "Logo Title Text 1")

![alt text](http://i.imgur.com/fkEbXXX.png "Logo Title Text 1")
단순화된 1 차원 영역에 나타낸 모델입니다. (a) 도메인 A 의 두 모드가 도메인 B 의 두 모드에 매핑되는 이상적인 구조, (b) GAN 모델이 실패한 경우, (c) GAN 에서의 재구성 실패



2 쌍의 모델이 하나의 도메인으로부터 다른 것으로 매핑하는 것과 
재구성을 위한 반대 과정도 학습합니다. 

두 개의 모델은 동시에 훈련됩니다.

- 총 4 개의 생성기
- 2 개의 판별기

GAB 두 개의 생성기와
GBA 두 개의 생성기는 파라미터를 공유하고, 생성된 이미지인
xBA 와 xAB 는 판별기 LDA 와
LDB 에 적용됩니다.


In [17]:
""" 네트워크 """

#2 쌍의 모델이 하나의 도메인으로부터 다른 것으로 매핑하는 것과
#재구성을 위한 반대 과정도 학습합니다.
#두 개의 모델은 동시에 훈련됩니다.
#GAB 두 개의 생성기와
#GBA 두 개의 생성기는 파라미터를 공유하고, 생성된 이미지인
#xBA 와 xAB 는 판별기 LDA 와
#LDB 에 적용됩니다.



#2 개의 생성기
def generative_network(z, input_dim, n_layer, n_hidden, eps_dim):
    with tf.variable_scope("generative"):
        h = z
        #repeat 을 사용해 동일한 연산을 반복적으로 실행합니다.
        #다수의 완전 연결 층
        h = slim.repeat(h, n_layer, slim.fully_connected, n_hidden, activation_fn=tf.nn.relu)
        x = slim.fully_connected(h, input_dim, activation_fn=None, scope="p_x")
    return x


def inference_network(x, latent_dim, n_layer, n_hidden, eps_dim):
    with tf.variable_scope("inference"):
        h = x
        h = slim.repeat(h, n_layer, slim.fully_connected, n_hidden, activation_fn=tf.nn.relu)
        z = slim.fully_connected(h, latent_dim, activation_fn=None, scope="q_z")
    return z


#2 개의 판별기
def data_network_x(x, n_layers=2, n_hidden=256, activation_fn=None):
    """Approximate x log data density."""
    h = tf.concat(x, 1)
    with tf.variable_scope('discriminator_x'):
        h = slim.repeat(h, n_layers, slim.fully_connected, n_hidden, activation_fn=tf.nn.relu)
        log_d = slim.fully_connected(h, 1, activation_fn=activation_fn)
    return tf.squeeze(log_d, squeeze_dims=[1]) #텐서의 모양으로부터
    #크기 1 의 차원을 제거합니다.



def data_network_z(z, n_layers=2, n_hidden=256, activation_fn=None):
    """Approximate z log data density."""
    h = tf.concat(z, 1)
    with tf.variable_scope('discriminator_z'):
        h = slim.repeat(h, n_layers, slim.fully_connected, n_hidden, activation_fn=tf.nn.relu)
        log_d = slim.fully_connected(h, 1, activation_fn=activation_fn)
    return tf.squeeze(log_d, squeeze_dims=[1])

In [18]:
""" Construct model and training ops """
tf.reset_default_graph()

#데이터 1 의 입력
x = tf.placeholder(tf.float32, shape=(batch_size, input_dim))
#데이터 2 의 입력
z = tf.placeholder(tf.float32, shape=(batch_size, latent_dim))

# 2 개의 생성기 - 인코더
p_x = generative_network(z, input_dim , n_layer_gen, n_hidden_gen, eps_dim)
q_z = inference_network(x, latent_dim, n_layer_inf, n_hidden_inf, eps_dim)

#로지트 함수는 시그모이드 로지스틱 함수의 역함수입니다

#2 개의 판별기
decoder_logit_x = data_network_x(p_x, n_layers=n_layer_disc, n_hidden=n_hidden_disc)
encoder_logit_x = graph_replace(decoder_logit_x, {p_x: x})

decoder_logit_z = data_network_z(q_z, n_layers=n_layer_disc, n_hidden=n_hidden_disc)
encoder_logit_z = graph_replace(decoder_logit_z, {q_z: z})

#소프트플러스를 계산합니다: log(exp(특성) + 1)
#손실 계산을 위한 활성값
encoder_sigmoid_x = tf.nn.softplus(encoder_logit_x)
decoder_sigmoid_x = tf.nn.softplus(decoder_logit_x)
encoder_sigmoid_z = tf.nn.softplus(encoder_logit_z)
decoder_sigmoid_z = tf.nn.softplus(decoder_logit_z)

INFO:tensorflow:Copying op: concat
INFO:tensorflow:Copying op: discriminator_x/Repeat/fully_connected_1/MatMul
INFO:tensorflow:Copying op: discriminator_x/Repeat/fully_connected_1/BiasAdd
INFO:tensorflow:Copying op: discriminator_x/Repeat/fully_connected_1/Relu
INFO:tensorflow:Copying op: discriminator_x/Repeat/fully_connected_2/MatMul
INFO:tensorflow:Copying op: discriminator_x/Repeat/fully_connected_2/BiasAdd
INFO:tensorflow:Copying op: discriminator_x/Repeat/fully_connected_2/Relu
INFO:tensorflow:Copying op: discriminator_x/fully_connected/MatMul
INFO:tensorflow:Copying op: discriminator_x/fully_connected/BiasAdd
INFO:tensorflow:Copying op: Squeeze
INFO:tensorflow:Finalizing op: concat
INFO:tensorflow:Finalizing op: discriminator_x/Repeat/fully_connected_1/MatMul
INFO:tensorflow:Finalizing op: discriminator_x/Repeat/fully_connected_1/BiasAdd
INFO:tensorflow:Finalizing op: discriminator_x/Repeat/fully_connected_1/Relu
INFO:tensorflow:Finalizing op: discriminator_x/Repeat/fully_connec

In [20]:
#손실 함수

#양 판별기의 손실
decoder_loss = decoder_sigmoid_x + decoder_sigmoid_z
encoder_loss = encoder_sigmoid_x + encoder_sigmoid_z

#판별기들의 손실 조합
disc_loss = tf.reduce_mean(  encoder_loss ) - tf.reduce_mean( decoder_loss)

#2 개의 생성기 (디코더)
rec_z = inference_network(p_x, latent_dim, n_layer_inf, n_hidden_inf, eps_dim )
rec_x = generative_network(q_z, input_dim , n_layer_gen, n_hidden_gen,  eps_dim )

#생성기의 손실을 계산합니다
#오류 제곱합의 손실
cost_z = tf.reduce_mean(tf.pow(rec_z - z, 2))
cost_x = tf.reduce_mean(tf.pow(rec_x - x, 2))
#판별기의 손실과 생성기의 손실을 결합합니다
adv_loss = tf.reduce_mean(  decoder_loss ) 
gen_loss = 1*adv_loss + 1.*cost_x  + 1.*cost_z

#이름 중 아래를 포함한 변수를 찾습니다
qvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "inference")
pvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "generative")
dvars_x = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "discriminator_x")
dvars_z = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "discriminator_z")

#adam (경사 하강법) 을 사용해 최적화합니다
opt = tf.train.AdamOptimizer(1e-4, beta1=0.5)

#생성기의 손실을 최소화합니다
train_gen_op =  opt.minimize(gen_loss, var_list=qvars + pvars)

#판별기의 손실을 최소화합니다
train_disc_op = opt.minimize(disc_loss, var_list=dvars_x + dvars_z)

In [21]:
""" training """
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

FG = []
FD = []

#각 epoch (상태 바의 로그 기록) 마다 반복
for epoch in tqdm( range(n_epoch), total=n_epoch):
    #양 데이터 세트의 샘플
    X_dataset, Z_dataset= shuffle(X_dataset, Z_dataset)

    #데이터의 각 x 와 y 에 대해 반복
    for xmb, zmb in iter_data(X_dataset, Z_dataset, size=batch_size):
        
        #손실 함수를 최소화합니다
        for _ in range(1):
            f_d, _ = sess.run([disc_loss, train_disc_op], feed_dict={x: xmb, z:zmb})
        for _ in range(5):
            #생성기의 손실을 만드는 3 개의 요소
            f_g, _ = sess.run([[adv_loss, cost_x, cost_z], train_gen_op], feed_dict={x: xmb, z:zmb})

        FG.append(f_g)
        FD.append(f_d)


 19%|█▊        | 187/1000 [00:50<03:22,  4.02it/s]

KeyboardInterrupt: 

In [ ]:
""" plot the results """

n_viz = 1
imz = np.array([]); rmz = np.array([]); imx = np.array([]); rmx = np.array([]);
for _ in range(n_viz):
    for xmb, zmb in iter_data(X_np_data, Z_np_data, size=batch_size):
        temp_imz = sess.run(q_z, feed_dict={x: xmb, z:zmb})
        imz = np.vstack([imz, temp_imz]) if imz.size else temp_imz

        temp_rmz = sess.run(rec_z, feed_dict={x: xmb, z:zmb})
        rmz = np.vstack([rmz, temp_rmz]) if rmz.size else temp_rmz

        temp_imx = sess.run(p_x, feed_dict={x: xmb, z:zmb})
        imx = np.vstack([imx, temp_imx]) if imx.size else temp_imx

        temp_rmx = sess.run(rec_x, feed_dict={x: xmb, z:zmb})
        rmx = np.vstack([rmx, temp_rmx]) if rmx.size else temp_rmx

## 추론된 z 의 가장자리값
fig_mz, ax = plt.subplots(nrows=1, ncols=1, figsize=(4.5, 4.5))
ll = np.tile(X_labels, (n_viz))
ax.scatter(imz[:, 0], imz[:, 1], c=cm.Set1(ll.astype(float)/input_dim/2.0),
        edgecolor='none', alpha=0.5)
ax.set_xlim(-3, 3); ax.set_ylim(-3.5, 3.5)
ax.set_xlabel('$z_1$'); ax.set_ylabel('$z_2$')
ax.axis('on')
plt.savefig(result_dir + 'inferred_mz.pdf', transparent=True, bbox_inches='tight')

##  재구성된 z 의 값
fig_pz, ax = plt.subplots(nrows=1, ncols=1, figsize=(4.5, 4.5))
ll = np.tile(Z_labels, (n_viz))
ax.scatter(rmz[:, 0], rmz[:, 1], c=cm.Set1(ll.astype(float)/input_dim/2.0),
           edgecolor='none', alpha=0.5)
ax.set_xlim(-3, 3); ax.set_ylim(-3.5, 3.5)
ax.set_xlabel('$z_1$'); ax.set_ylabel('$z_2$')
ax.axis('on')
plt.savefig(result_dir + 'reconstruct_mz.pdf', transparent=True, bbox_inches='tight')

## 추론된 x 의 가장자리값
fig_pz, ax = plt.subplots(nrows=1, ncols=1, figsize=(4.5, 4.5))
ll = np.tile(Z_labels, (n_viz))
ax.scatter(imx[:, 0], imx[:, 1], c=cm.Set1(ll.astype(float)/input_dim/2.0),
        edgecolor='none', alpha=0.5)
ax.set_xlim(-3, 3); ax.set_ylim(-3.5, 3.5)
ax.set_xlabel('$x_1$'); ax.set_ylabel('$x_2$')
ax.axis('on')
plt.savefig(result_dir + 'inferred_mx.pdf', transparent=True, bbox_inches='tight')

##  재구성된 x 의 값
fig_mx, ax = plt.subplots(nrows=1, ncols=1, figsize=(4.5, 4.5))
ll = np.tile(X_labels, (n_viz))
ax.scatter(rmx[:, 0], rmx[:, 1], c=cm.Set1(ll.astype(float)/input_dim/2.0),
           edgecolor='none', alpha=0.5)
ax.set_xlim(-3, 3); ax.set_ylim(-3.5, 3.5)
ax.set_xlabel('$x_1$'); ax.set_ylabel('$x_2$')
ax.axis('on')
plt.savefig(result_dir + 'reconstruct_mx.pdf', transparent=True, bbox_inches='tight')

## 곡선을 학습합니다
fig_curve, ax = plt.subplots(nrows=1, ncols=1, figsize=(4.5, 4.5))
ax.plot(FD, label="Discriminator")
ax.plot(np.array(FG)[:,0], label="Generator")
ax.plot(np.array(FG)[:,1], label="Reconstruction x")
ax.plot(np.array(FG)[:,2], label="Reconstruction Z")
plt.xlabel('Iteration')
plt.xlabel('Loss')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax.axis('on')
plt.savefig(result_dir + 'learning_curves.pdf', bbox_inches='tight')
